In [693]:
# ruff: noqa: F401, E402
import math
import sys
from bisect import bisect_left, bisect_right
from collections import Counter, deque
from functools import cmp_to_key, lru_cache, reduce
from heapq import heapify, heappop, heappush, heappushpop, nsmallest
from importlib import reload
from itertools import (
    accumulate,
    chain,
    combinations,
    islice,
    pairwise,
    permutations,
    product,
    starmap,
    tee,
)
from math import comb, factorial, log, sqrt
from operator import add, and_, contains, indexOf, itemgetter, neg, or_, xor
from pprint import pprint
from typing import Callable, Iterable, List, Tuple

import matplotlib.pyplot as plt
import numpy as np

import arrays
import graphs
import lists
import mathematics
import matrix
import parsing
import search
import sequences
import sets
import strings
import trees

for m in (
    arrays,
    graphs,
    lists,
    mathematics,
    matrix,
    parsing,
    search,
    sequences,
    sets,
    strings,
    trees,
):
    reload(m)


from parsing import format_binop_expression, parse_binop_expression
from sequences import find_if
from strings import StringView

print()


In [727]:
def distinct_palindrome_substrings(s: str) -> int:
    "Return the count of distinct palindromes within `s`."
    # Uses Manacher's algorithm.
    # Runs in O(N*N) :
    #   O(N) for the Manacher's algorithm.
    #   O(N) for cutting the string into pieces and a hash-set operation.
    n = len(s) * 2
    if n == 0:
        return 0
    pals = set()
    lps = [StringView("", 0, 0)] * n
    lps[1] = StringView(s, 0, 1)  # View of the first palindrome
    c = 1  # center of the outer palindrome (in s*2)
    r = 2  # right border of the outer palindrome (in s*2)
    for i in range(2, n):
        if i < r:
            # Use the mirror length if `i` is between `c` and `r`.
            v = lps[2 * c - i]
            l = min(len(v), r - i)
            h = l == len(v) and hash(v)
        else:
            # Start from the middle.
            l, h = 0, None

        v = StringView(s, (i - l) // 2, (i + l + 1) // 2, h=h)
        while 0 < v.start and v.stop < len(s) and s[v.start - 1] == s[v.stop]:
            v = v.expand(-1, 1)
            pals.add(v)
        # Store the value.
        lps[i] = v
        # If the current palindrome expanded beyond the
        # previous one, replace the old by this one.
        if i + len(v) > r:
            c = i
            r = i + len(v)
    return len(pals) + len(set(iter(s)))


In [728]:
distinct_palindrome_substrings("abaaaaabba")


10

In [729]:
distinct_palindrome_substrings("cdgdcdgdcdggdcdggd")


18

In [730]:
distinct_palindrome_substrings("mdnvznwlylygvstwarpibrfgvdhkdcrlmfgqweveqyo")


26